In [1]:
import pandas as pd
import numpy as np

### Join census tracts and check-in dataset 

In [2]:
census_tracts = pd.read_csv('../data/processed/census_tracts_per_checkin.csv')
checkin_dataset = pd.read_csv('../data/raw/foursquare-nyc-check-ins/dataset_TSMC2014_NYC.csv')
checkin_dataset['censusTract'] = census_tracts['census_tract']

In [3]:
checkin_dataset.head()

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp,censusTract
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012,36061003300
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012,36085001800
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012,36081065900
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,Tue Apr 03 18:02:41 +0000 2012,36061007200
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,Tue Apr 03 18:03:00 +0000 2012,36061005600


### Filter tracts with no income for census tract

In [4]:
labels_path = '../data/raw/new-york-city-census-data/'
demographics = pd.read_csv(labels_path + 'nyc_census_tracts.csv')
print 'total census tracts:', len(demographics)
demographics = demographics[demographics['Income'].notnull()]
print 'excluded null:', len(demographics)
demographics.head(1)

total census tracts: 2167
excluded null: 2101


,CensusTract,County,Borough,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
1,36005000200,Bronx,Bronx,5403,2659,2744,75.8,2.3,16.0,0.0,...,2.9,0.0,0.0,43.0,2308,80.8,16.2,2.9,0.0,7.7


In [5]:
examples = pd.DataFrame()
examples['census_tract'] = demographics['CensusTract']
examples['label'] = demographics['Income']
examples = examples[examples['label'].notnull()]
# examples = examples.head(100) # debug

In [6]:
# # use this cell to test features
# checkins = checkin_dataset[checkin_dataset['censusTract'] == 36005022401]
# curr_example = examples[examples['census_tract'] == 36005022401]  

In [7]:
# helper function to strip all non-alphanumericc characters to form feature keys
import re
def keyify_string(s):
    return re.sub('[^0-9a-zA-Z]+', '_', s).lower()

#### feature: total checkins per census tract

In [8]:
def get_num_checkins_key():
        return 'num_checkins'

def add_num_checkins(checkins, idx):    
    examples.loc[idx, get_num_checkins_key()] = len(checkins)

#### feature: num checkins per venue category

In [9]:
def get_num_category_key(s):
    return 'num_' + keyify_string(s)

def add_num_category(checkins, idx):
    checkins_per_category = checkins.groupby('venueCategory').utcTimestamp.nunique()
    for k, v in checkins_per_category.iteritems():
        examples.loc[idx, get_num_category_key(k)] = v
        
possible_categories = [get_num_category_key(k) for k in checkin_dataset.venueCategory.unique()]

#### feature: num checkins per weekday

In [10]:
import dateutil.parser
import datetime
from collections import Counter

def get_num_weekday_key(weekday): 
    return 'num_' + keyify_string(weekday)

def add_num_weekday(checkins, idx):
    datetime_objects = [dateutil.parser.parse(ts) + datetime.timedelta(offset)  \
                        for ts, offset in zip(checkins['utcTimestamp'], checkins['timezoneOffset'])]
    
    weekdays = [datetime.datetime.strftime(dt, '%a') for dt in datetime_objects]
    c = Counter(weekdays)
    for k, v in zip(c.keys(), c.values()):
        examples.loc[idx, get_num_weekday_key(k)] = v
    
possible_weekdays = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

#### feature: num checkins per 4-hour time bucket

In [11]:
def time_buckets(datetime_object):
    hour = int(datetime.datetime.strftime(datetime_object, '%H'))
    if hour <= 4: return '12am_4am'
    if hour <= 8: return '4am_8am'
    if hour <= 12: return '8am_12pm'
    if hour <= 16: return '12pm_4pm'
    if hour <= 20: return '4pm_8pm'
    else: return '8pm_12am'

def get_num_time_buckets_key(bucket):
    return 'num_' + keyify_string(bucket)

def add_num_time_buckets(checkins, idx):
    datetime_objects = [dateutil.parser.parse(ts) + datetime.timedelta(offset)  \
                        for ts, offset in zip(checkins['utcTimestamp'], checkins['timezoneOffset'])]

    hours = [time_buckets(dt) for dt in datetime_objects]
    c = Counter(hours)
    for k, v in zip(c.keys(), c.values()):
        examples.loc[idx, get_num_time_buckets_key(k)] = v
        
possible_time_buckets = ['12am_4am', '4am_8am', '8am_12pm', '12pm_4pm', '4pm_8pm', '8pm_12am']

#### feature: num checkins per 4-hour time bucket per weekday

In [12]:
def get_num_time_buckets_per_weekday(bucket, weekday):
    return 'num_' + bucket + '_' + keyify_string(weekday)

def add_num_time_buckets_per_weekday(checkins, idx):
    datetime_objects = [dateutil.parser.parse(ts) + datetime.timedelta(offset)  \
                        for ts, offset in zip(checkins['utcTimestamp'], checkins['timezoneOffset'])]

    buckets_per_weekday = [(time_buckets(dt), datetime.datetime.strftime(dt, '%a')) 
             for dt in datetime_objects]

    c = Counter(buckets_per_weekday)
    for k, v in zip(c.keys(), c.values()):
        bucket, weekday = k
        examples.loc[idx, get_num_time_buckets_per_weekday(bucket, weekday)] = v

#### feature: num checkins per 4-hour time bucket per venue category

In [13]:
def get_num_time_buckets_per_category(bucket, category):
    return 'num_' + bucket + '_' + keyify_string(category)

def add_num_time_buckets_per_category(checkins, idx):
    datetime_objects_per_category = [(dateutil.parser.parse(ts) + datetime.timedelta(offset), category)  \
                        for ts, offset, category in zip(checkins['utcTimestamp'], checkins['timezoneOffset'], 
                                            checkins['venueCategory'])]

    buckets_per_category = [(time_buckets(dt), category) 
             for dt, category in datetime_objects_per_category]

    c = Counter(buckets_per_category)
    for k, v in zip(c.keys(), c.values()):
        buckets, categories = k
        examples.loc[idx, get_num_time_buckets_per_category(buckets, categories)] = v

#### feature: num checkins per weekday per venue category

In [14]:
def get_num_time_buckets_per_week_per_category(bucket, weekday, category):
    return 'num_' + bucket + '_' + keyify_string(weekday) + '_ '+ keyify_string(category)

def add_num_time_buckets_per_week_per_category(checkins, idx):
    datetime_objects_per_category = [(dateutil.parser.parse(ts) + datetime.timedelta(offset), category)  \
                            for ts, offset, category in zip(checkins['utcTimestamp'], checkins['timezoneOffset'], 
                                                checkins['venueCategory'])]

    buckets_per_weekday_per_category = [(time_buckets(dt), datetime.datetime.strftime(dt, '%a'), category) 
             for dt, category in datetime_objects_per_category]

    c = Counter(buckets_per_weekday_per_category)
    for k, v in zip(c.keys(), c.values()):
        buckets, weekday, categories = k
        examples.loc[idx, get_num_time_buckets_per_week_per_category(buckets, weekday, categories)] = v

In [15]:
examples.head()

,census_tract,label
1,36005000200,72034.0
2,36005000400,74836.0
3,36005001600,32312.0
4,36005001900,37936.0
5,36005002000,18086.0


### test/select features

In [16]:
%%time 

import sys

no_checkins = 0
total_checkins = 0
for idx, d in examples['census_tract'].iteritems():
    # show progress 
    sys.stdout.write('\r' + str(idx+1) + ' / ' + str(len(examples)) + ' examples processed')
    sys.stdout.flush()
    
    # compute all checkins in current census tract
    checkins_in_census_tract = checkin_dataset[checkin_dataset['censusTract'] == d]
    
    # compute metadata
    total_checkins += len(checkins_in_census_tract)
    if len(checkins_in_census_tract) == 0: 
        no_checkins += 1
    
    # add features
    add_num_checkins(checkins_in_census_tract, idx)
    add_num_category(checkins_in_census_tract, idx)
    add_num_weekday(checkins_in_census_tract, idx)
    add_num_time_buckets(checkins_in_census_tract, idx)
#     add_num_time_buckets_per_category(checkins_in_census_tract, idx)
#     add_num_time_buckets_per_weekday(checkins_in_census_tract, idx)
#     add_num_time_buckets_per_week_per_category(checkins_in_census_tract, idx)

2166 / 2101 examples processedCPU times: user 2min 8s, sys: 5.82 s, total: 2min 14s
Wall time: 2min 31s


In [17]:
print 'no checkin data associated with # businesses:', no_checkins
print total_checkins

no checkin data associated with # businesses: 220
174417


In [18]:
print examples.shape
examples.head()

(2101, 266)


,census_tract,label,num_checkins,num_park,num_fri,num_4pm_8pm,num_thu,num_12pm_4pm,num_bus_station,num_home_private_,...,num_swiss_restaurant,num_peruvian_restaurant,num_gluten_free_restaurant,num_funeral_home,num_law_school,num_pet_service,num_recycling_facility,num_internet_cafe,num_motorcycle_shop,num_planetarium
1,36005000200,72034.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36005000400,74836.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,36005001600,32312.0,76.0,NaN,41.0,16.0,1.0,17.0,6.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36005001900,37936.0,77.0,NaN,1.0,33.0,5.0,30.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,36005002000,18086.0,4.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### test features with nb model

In [19]:
# split incomes into quartiles
income_mapping = {
    '9829 <= x <= 39457': (9829, 39457),
    '39458 <= x <= 54595': (39458, 54595), 
    '54596 <= x <= 73345': (54596, 73345), 
    '73346 <= x <= 244375': (73346, 244375),
}

# import pdb; pdb.set_trace()
for (imk, imv) in income_mapping.iteritems():
    examples.loc[(examples['label'] >= imv[0]) & (examples['label'] <= imv[1]), 'label_bucket'] = imk

In [20]:
# fill .nans with 0 
examples = examples.fillna(0)

# move new col to beginning of matrix
cols = list(examples)
cols.insert(2, cols.pop(cols.index('label_bucket')))
examples = examples[cols]
examples.head()

,census_tract,label,label_bucket,num_checkins,num_park,num_fri,num_4pm_8pm,num_thu,num_12pm_4pm,num_bus_station,...,num_swiss_restaurant,num_peruvian_restaurant,num_gluten_free_restaurant,num_funeral_home,num_law_school,num_pet_service,num_recycling_facility,num_internet_cafe,num_motorcycle_shop,num_planetarium
1,36005000200,72034.0,54596 <= x <= 73345,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,36005000400,74836.0,73346 <= x <= 244375,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,36005001600,32312.0,9829 <= x <= 39457,76.0,0.0,41.0,16.0,1.0,17.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,36005001900,37936.0,9829 <= x <= 39457,77.0,0.0,1.0,33.0,5.0,30.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,36005002000,18086.0,9829 <= x <= 39457,4.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [22]:
# data = pd.read_csv('../common/data/data.csv')
x_train = examples.as_matrix()[:, 3:].astype(int) # take all cols after first 2
y_train = examples['label']

In [23]:
y_train = examples['label_bucket']

In [24]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(x_train, y_train).predict(x_train)
print("Number of mislabeled points out of a total %d points : %d" % (x_train.shape[0],(y_train != y_pred).sum()))
print 1.0*(y_train == y_pred).sum()/ len(y_pred)

Number of mislabeled points out of a total 2101 points : 1292
0.38505473584


In [25]:
examples.shape

(2101, 267)

### uncomment to save file with feature dimensions (`examples.shape[1]-3`) in filename

In [26]:
# uncomment to save to file
# examples.to_csv('../data/processed/cat_264.csv', index=False)